In [ ]:
########## Système
import datetime

########## DataFrames et calculs
import pandas as pd
import numpy as np

######### Machine Learning
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import f1_score, mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from tqdm import tqdm, trange

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# 0. Configuration

In [ ]:
# Data loading
df =  pd.read_pickle("/content/data.pkl")
df

,Perf,Perf.1,Perf.2,Perf.3,Perf.4,Perf.5,Perf.6,Perf.7,Perf.8,Perf.9,...,CA to GDP NEW.11,CA to GDP NEW.12,CA to GDP NEW.13,CA to GDP NEW.14,CA to GDP NEW.15,CA to GDP NEW.16,CA to GDP NEW.17,CA to GDP NEW.18,Perf JP,Perf Barclays
Date,,,,,,,,,,,,,,,,,,,,,
2006-07-31,0.073549,0.041356,0.010963,0.014121,0.038419,0.013921,0.031179,0.032397,0.019341,0.030846,...,0.081358,0.020513,-0.042853,-0.085026,0.117494,-0.044676,-0.046059,0.001892,0.031179,0.016727
2006-08-31,0.076104,0.028205,0.019960,0.017543,0.024432,0.011671,0.025704,0.024110,0.018641,0.026751,...,0.081358,0.026107,-0.042853,-0.085026,0.117494,-0.044676,-0.046059,0.001892,0.025704,0.018631
2006-09-30,-0.006634,0.005193,0.007675,0.009450,0.007837,0.011568,0.005922,0.016214,0.010281,0.005319,...,0.081358,0.032634,-0.042853,-0.085026,0.117494,-0.044676,-0.046059,0.001892,0.005922,0.009444
2006-10-31,0.081491,0.017419,0.008479,0.009223,0.019092,0.006176,0.019565,0.014994,0.011170,0.014165,...,0.081358,0.038589,-0.043821,-0.090806,0.112051,-0.037676,-0.052566,-0.005294,0.019565,0.017023
2006-11-30,0.019146,0.018172,0.014173,0.013859,0.009806,0.014391,0.011861,0.021385,0.017301,0.010380,...,0.081358,0.040384,-0.043821,-0.090806,0.112051,-0.037676,-0.052566,-0.005294,0.011861,0.020040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,-0.032346,0.017010,0.027456,0.012213,0.021582,0.033791,0.018730,0.019748,0.023292,0.026695,...,0.009678,-0.024253,0.017848,-0.071532,0.024644,-0.063184,-0.033924,-0.095780,0.018028,0.014878
2024-06-28,-0.026710,0.001664,0.011629,0.007890,0.011754,-0.028023,0.007315,0.010072,0.004276,-0.000147,...,0.009678,-0.023795,0.017848,-0.070677,0.024644,-0.063184,-0.028949,-0.095217,0.006157,0.003994
2024-07-31,0.008526,0.025440,0.020990,0.017575,0.019130,0.025057,0.020073,0.016181,0.025525,0.024013,...,0.019618,-0.019602,0.015937,-0.068709,0.031188,-0.068244,-0.027288,-0.106485,0.018677,0.019585


# 1. Data preprocessing


In [ ]:
liste_colonnes = df.columns.tolist()
print(liste_colonnes)

['Perf', 'Perf.1', 'Perf.2', 'Perf.3', 'Perf.4', 'Perf.5', 'Perf.6', 'Perf.7', 'Perf.8', 'Perf.9', 'Perf.10', 'Perf.11', 'Perf.12', 'Perf.13', 'Perf.14', 'Perf.15', 'Perf.16', 'Perf.17', 'Perf.18', 'Beta 1', 'Beta 1.1', 'Beta 1.2', 'Beta 1.3', 'Beta 1.4', 'Beta 1.5', 'Beta 1.6', 'Beta 1.7', 'Beta 1.8', 'Beta 1.9', 'Beta 1.10', 'Beta 1.11', 'Beta 1.12', 'Beta 1.13', 'Beta 1.14', 'Beta 1.15', 'Beta 1.16', 'Beta 1.17', 'Beta 1.18', 'Beta 2', 'Beta 2.1', 'Beta 2.2', 'Beta 2.3', 'Beta 2.4', 'Beta 2.5', 'Beta 2.6', 'Beta 2.7', 'Beta 2.8', 'Beta 2.9', 'Beta 2.10', 'Beta 2.11', 'Beta 2.12', 'Beta 2.13', 'Beta 2.14', 'Beta 2.15', 'Beta 2.16', 'Beta 2.17', 'Beta 2.18', 'Rating', 'Rating.1', 'Rating.2', 'Rating.3', 'Rating.4', 'Rating.5', 'Rating.6', 'Rating.7', 'Rating.8', 'Rating.9', 'Rating.10', 'Rating.11', 'Rating.12', 'Rating.13', 'Rating.14', 'Rating.15', 'Rating.16', 'Rating.17', 'Rating.18', 'Variation Devise', 'Variation Devise.1', 'Variation Devise.2', 'Variation Devise.3', 'Variation 

In [ ]:
features_sans_point = [col for col in df.columns if '.' not in col]
len(features_sans_point)
features_sans_point

['Perf',
 'Beta 1',
 'Beta 2',
 'Rating',
 'Variation Devise',
 'Variation CB_Rate',
 'Performance 6M',
 'Performance 12M',
 'CPI',
 'R',
 'Variation CPI',
 'Equi',
 'Equi Perf6M',
 'Equi Perf12M',
 'GDP',
 'Yield',
 'Var Yield 5 ans',
 'Trade Balance to GDP',
 'Import / Res Change',
 'Debt to GDP',
 'External Debt to Tot Reserve',
 '(External Debt -FX Res) to GDP',
 'Variation CB_RateNEW',
 'R_NEW',
 'CPI_NEW',
 'Variation CPI_NEW',
 'Import / Tot Res NEW',
 'Debt to GDP_NEW',
 'External Debt to Tot Reserve NEW',
 '(External Debt - Tot Res) to GDP NEW',
 'CA to GDP NEW',
 'Perf JP',
 'Perf Barclays']

In [ ]:
# Create a dictionary to hold dataframes for each country, including the first country with no suffix
country_data = {'0': [col for col in df.columns if '.' not in col]}

# Extract the suffixes and group columns by country
for col in df.columns:
    suffix = col.split('.')[-1]
    if suffix.isdigit():  # Check if the suffix is a number
        if suffix not in country_data:
            country_data[suffix] = []
        country_data[suffix].append(col)

# Convert lists of columns into sub-dataframes for each country
for key in country_data.keys():
    country_data[key] = df[country_data[key]]

# Generate summary statistics for each country's dataframe
country_summaries = {key: df.describe() for key, df in country_data.items()}

# Example: Print summaries for country "0" (no suffix) and country "1" (first suffix)
country_summaries['0']

,Perf,Beta 1,Beta 2,Rating,Variation Devise,Variation CB_Rate,Performance 6M,Performance 12M,CPI,R,...,R_NEW,CPI_NEW,Variation CPI_NEW,Import / Tot Res NEW,Debt to GDP_NEW,External Debt to Tot Reserve NEW,(External Debt - Tot Res) to GDP NEW,CA to GDP NEW,Perf JP,Perf Barclays
count,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000,...,219.000000,219.00000,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000
mean,0.010116,2.796936,2.499624,0.199543,0.427203,3.773687,0.058772,0.107219,27.917260,27.842893,...,29.443635,41.51984,11.405205,8.930831,65.512384,5.000095,0.326194,-0.008000,0.004788,0.005593
std,0.099976,2.234291,2.201498,0.127430,0.608490,18.355548,0.258161,0.382601,20.468678,19.298286,...,27.234979,51.31151,33.007597,3.227119,26.018702,2.057784,0.120760,0.023392,0.026615,0.029800
min,-0.515285,-0.058045,-2.155316,0.000000,-0.039985,-78.000000,-0.594773,-0.670172,5.320000,7.687500,...,6.500000,5.32000,-24.330000,3.556131,37.550000,2.698053,0.180189,-0.070180,-0.160329,-0.186435
25%,-0.036336,1.503111,1.453870,0.100000,0.080285,-1.218750,-0.076775,-0.134139,12.650000,12.937500,...,9.500000,12.97000,-3.470000,6.368474,43.590000,3.370912,0.215116,-0.024508,-0.005246,-0.008478
50%,0.011123,2.284449,2.082932,0.250000,0.222680,1.562500,0.037985,0.064744,15.160000,21.125000,...,20.000000,26.16000,3.600000,8.345322,55.080000,4.482702,0.305401,-0.007883,0.007380,0.006500
75%,0.060884,3.536241,2.898337,0.300000,0.561417,6.343750,0.168927,0.338048,54.075000,38.000000,...,38.000000,48.00000,15.196000,10.771680,84.685000,6.228741,0.399168,0.012223,0.018022,0.019799
max,0.337963,16.150493,20.296886,0.350000,3.564252,58.000000,1.319564,1.682662,76.620000,97.000000,...,133.000000,289.40000,183.600000,17.654149,154.536000,12.837515,0.657674,0.027701,0.075913,0.108882


# 2. Modeling

## Régression linéaire

In [ ]:
# Sélection des features et de la target pour le pays '0'
X = country_data['0'].drop('Perf', axis=1)  # Toutes les colonnes sauf la target
y = country_data['0']['Perf']               # Colonne target

# Division des données en ensembles d'apprentissage et de test (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Création et entraînement du modèle de régression linéaire
model = LinearRegression() # explorer possibilité NLP / transformée rapide de fourier (library darts) / pour autre models hyperparam
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Calcul de l'erreur quadratique moyenne pour évaluer le modèle (4 métriques possibles voir lien réunion)
mse = mean_squared_error(y_test, y_pred)
print(f"L'erreur quadratique moyenne (MSE) du modèle est : {mse:.4f}") # ou RMSE

# Récupération du coefficient de détermination R^2
r2 = model.score(X_test, y_test)
print(f"Le coefficient de détermination R^2 du modèle est : {r2:.4f}")

L'erreur quadratique moyenne (MSE) du modèle est : 0.0059
Le coefficient de détermination R^2 du modèle est : -0.0429


1. Erreur Quadratique Moyenne (MSE): Le MSE est une mesure de la qualité d’un estimateur — il est toujours non négatif, et des valeurs plus petites indiquent de meilleures performances. Un MSE de 0.0059 suggère que les prédictions du modèle sont relativement proches des valeurs réelles en moyenne, compte tenu de l’échelle des données. Cependant, l’interprétation de cette valeur dépend de l’échelle des données de la target (‘Perf’). Si la plage de ‘Perf’ est large, un MSE de 0.0059 peut être considéré comme bon. S’il s’agit d’une échelle plus petite, cette valeur peut indiquer une performance moins impressionnante.


2. Coefficient de Détermination (R²): Le R² est une statistique qui donne une idée de la quantité de variance dans la variable dépendante que le modèle est capable d’expliquer. Il est généralement compris entre 0 et 1, où un R² proche de 1 indique que le modèle explique une grande partie de la variance de la target, tandis qu’un R² de 0 indique que le modèle n’explique pas mieux la variance que le modèle simple qui prédit la moyenne de la target. Un R² négatif, comme dans votre cas (-0.0429), suggère que le modèle est moins performant que le modèle de base qui prédit simplement la moyenne de la target. Cela peut indiquer plusieurs problèmes, comme un mauvais choix de variables, un manque de linéarité dans la relation entre les variables, ou un modèle qui est surajusté ou sous-ajusté.


In [ ]:
def run_linear_regression(country_data):
    results = []

    for country, df in country_data.items():
        if country == '0':
            perf_col = 'Perf'
        else:
            perf_col = f'Perf.{country}'

        if perf_col in df.columns:
            X = df.drop(perf_col, axis=1)
            y = df[perf_col]

            if X.empty or X.shape[1] == 0:
                print(f"Le pays {country} n'a pas suffisamment de features pour la régression.")
                continue

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

            model = LinearRegression()
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mse)

            results.append({
                'Country': country,
                'MSE': mse,
                'R2': r2,
                'MAE': mae,
                'RMSE': rmse
            })
        else:
            print(f"Le pays {country} n'a pas de colonne '{perf_col}' pour la prédiction.")

    # Conversion en DataFrame pour affichage
    results_df = pd.DataFrame(results)
    display(results_df)

    return results_df

# Appel de la fonction avec nos données
results_df = run_linear_regression(country_data)

,Country,MSE,R2,MAE,RMSE
0,0,0.005858,-0.042871,0.054973,0.076541
1,1,0.000816,-0.163837,0.021209,0.028559
2,2,0.000770,-0.194844,0.019713,0.027743
3,3,0.000220,-0.903777,0.011559,0.014844
4,4,0.000957,-0.080910,0.022855,0.030943
5,5,0.001200,0.063381,0.028199,0.034635
6,6,0.000461,-0.782755,0.015491,0.021482
7,7,0.001647,-0.446421,0.026546,0.040580
8,8,0.000493,-0.250164,0.015890,0.022208
9,9,0.000993,-0.364476,0.023150,0.031515


## Random Forest

In [ ]:
def run_random_forest(country_data):
    results = []

    for country, df in country_data.items():
        # Déterminer le nom correct de la colonne 'Perf'
        if country == '0':
            perf_col = 'Perf'
        else:
            perf_col = f'Perf.{country}'

        if perf_col in df.columns:
            X = df.drop(perf_col, axis=1)
            y = df[perf_col]

            if X.empty or X.shape[1] == 0:
                print(f"Le pays {country} n'a pas suffisamment de features pour la régression.")
                continue

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mse)

            results.append({
                'Country': country,
                'MSE': mse,
                'R2': r2,
                'MAE': mae,
                'RMSE': rmse
            })
        else:
            print(f"Le pays {country} n'a pas de colonne '{perf_col}' pour la prédiction.")

    results_df = pd.DataFrame(results)
    return results_df

# Appel de la fonction et affichage
rf_results_df = run_random_forest(country_data)
rf_results_df

,Country,MSE,R2,MAE,RMSE
0,0,0.004595,0.181970,0.051636,0.067789
1,1,0.000696,0.006866,0.018310,0.026381
2,2,0.000741,-0.150665,0.019716,0.027225
3,3,0.000137,-0.184457,0.009152,0.011709
4,4,0.001062,-0.199300,0.022173,0.032594
5,5,0.001464,-0.143013,0.026924,0.038261
6,6,0.000294,-0.135198,0.011333,0.017142
7,7,0.001542,-0.354111,0.022173,0.039263
8,8,0.000558,-0.414278,0.017122,0.023621
9,9,0.000818,-0.124007,0.020338,0.028604


In [ ]:
def run_optimized_random_forest(country_data):
    results = []

    for country, df in country_data.items():
        # Déterminer le nom correct de la colonne 'Perf'
        if country == '0':
            perf_col = 'Perf'
        else:
            perf_col = f'Perf.{country}'

        if perf_col in df.columns:
            X = df.drop(perf_col, axis=1)
            y = df[perf_col]

            if X.empty or X.shape[1] == 0:
                print(f"Le pays {country} n'a pas suffisamment de features pour la régression.")
                continue

            # Préparation de la grille de paramètres
            param_grid = {
                'n_estimators': [50, 100, 200, 300, 400, 500]
            }

            model = RandomForestRegressor(random_state=42)
            grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
            grid_search.fit(X, y)

            best_model = grid_search.best_estimator_
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

            best_model.fit(X_train, y_train)
            y_pred = best_model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mse)

            results.append({
                'Country': country,
                'Best n_estimators': grid_search.best_params_['n_estimators'],
                'MSE': mse,
                'R2': r2,
                'MAE': mae,
                'RMSE': rmse
            })
        else:
            print(f"Le pays {country} n'a pas de colonne '{perf_col}' pour la prédiction.")

    results_df = pd.DataFrame(results)
    return results_df

# Appel de la fonction et affichage
optimized_rf_results = run_optimized_random_forest(country_data)
optimized_rf_results

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates

,Country,Best n_estimators,MSE,R2,MAE,RMSE
0,0,200,0.004673,0.168240,0.051603,0.068356
1,1,200,0.000696,0.006550,0.018522,0.026385
2,2,50,0.000787,-0.221971,0.020103,0.028056
3,3,300,0.000141,-0.215124,0.009069,0.011859
4,4,200,0.001097,-0.238680,0.022363,0.033125
5,5,400,0.001461,-0.140923,0.026974,0.038226
6,6,200,0.000290,-0.121818,0.011131,0.017041
7,7,50,0.001547,-0.359221,0.022632,0.039337
8,8,50,0.000575,-0.457583,0.017084,0.023980
9,9,100,0.000818,-0.124007,0.020338,0.028604


## XGBoost

In [ ]:
def run_optimized_xgboost(country_data):
    results = []

    for country, df in country_data.items():
        # Déterminer le nom correct de la colonne 'Perf'
        if country == '0':
            perf_col = 'Perf'
        else:
            perf_col = f'Perf.{country}'

        if perf_col in df.columns:
            X = df.drop(perf_col, axis=1)
            y = df[perf_col]

            if X.empty or X.shape[1] == 0:
                print(f"Le pays {country} n'a pas suffisamment de features pour la régression.")
                continue

            # Préparation de la grille de paramètres
            param_grid = {
                #'n_estimators': [50, 100, 200, 300],
                'n_estimators': [50],
                #'max_depth': [3, 5, 7, 9],
                'max_depth': [3],
                #'learning_rate': [0.01, 0.05, 0.1]
                'learning_rate': [0.01]
            }

            model = XGBRegressor(random_state=42)
            grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
            grid_search.fit(X, y)

            best_model = grid_search.best_estimator_
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

            best_model.fit(X_train, y_train)
            y_pred = best_model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mse)

            results.append({
                'Country': country,
                'Best n_estimators': grid_search.best_params_['n_estimators'],
                'Best max_depth': grid_search.best_params_['max_depth'],
                'Best learning_rate': grid_search.best_params_['learning_rate'],
                'MSE': mse,
                'R2': r2,
                'MAE': mae,
                'RMSE': rmse
            })
        else:
            print(f"Le pays {country} n'a pas de colonne '{perf_col}' pour la prédiction.")

    results_df = pd.DataFrame(results)
    return results_df

# Appel de la fonction et affichage
optimized_xgb_results = run_optimized_xgboost(country_data)
optimized_xgb_results

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

,Country,Best n_estimators,Best max_depth,Best learning_rate,MSE,R2,MAE,RMSE
0,0,50,3,0.01,0.004331,0.228979,0.048236,0.065813
1,1,50,3,0.01,0.000659,0.059455,0.017528,0.025673
2,2,50,3,0.01,0.000646,-0.003312,0.018029,0.025422
3,3,50,3,0.01,0.000121,-0.047119,0.008374,0.011009
4,4,50,3,0.01,0.000915,-0.033248,0.020487,0.030253
5,5,50,3,0.01,0.001392,-0.086537,0.026126,0.037304
6,6,50,3,0.01,0.000260,-0.003632,0.010814,0.016118
7,7,50,3,0.01,0.001236,-0.085873,0.020851,0.035160
8,8,50,3,0.01,0.000476,-0.207762,0.015846,0.021828
9,9,50,3,0.01,0.000787,-0.081392,0.018955,0.028056


## LightGBM

In [ ]:
def run_optimized_lightgbm(country_data):
    results = []

    for country, df in country_data.items():
        # Déterminer le nom correct de la colonne 'Perf'
        if country == '0':
            perf_col = 'Perf'
        else:
            perf_col = f'Perf.{country}'

        if perf_col in df.columns:
            X = df.drop(perf_col, axis=1)
            y = df[perf_col]

            if X.empty or X.shape[1] == 0:
                print(f"Le pays {country} n'a pas suffisamment de features pour la régression.")
                continue

            # Préparation de la grille de paramètres
            param_grid = {
                #'n_estimators': [50, 100, 200, 300],
                'n_estimators': [50],
                #'max_depth': [3, 5, 7, 9],
                'max_depth': [3],
                #'learning_rate': [0.01, 0.05, 0.1],
                'learning_rate': [0.01],
                #'num_leaves': [31, 62, 124, 248]  # nombre de feuilles dans un arbre
                'num_leaves': [31]
            }

            model = LGBMRegressor(random_state=42)
            grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
            grid_search.fit(X, y)

            best_model = grid_search.best_estimator_
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

            best_model.fit(X_train, y_train)
            y_pred = best_model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mse)

            results.append({
                'Country': country,
                'Best n_estimators': grid_search.best_params_['n_estimators'],
                'Best max_depth': grid_search.best_params_['max_depth'],
                'Best learning_rate': grid_search.best_params_['learning_rate'],
                'Best num_leaves': grid_search.best_params_['num_leaves'],
                'MSE': mse,
                'R2': r2,
                'MAE': mae,
                'RMSE': rmse
            })
        else:
            print(f"Le pays {country} n'a pas de colonne '{perf_col}' pour la prédiction.")

    results_df = pd.DataFrame(results)
    return results_df

# Appel de la fonction et affichage
optimized_lgbm_results = run_optimized_lightgbm(country_data)
optimized_lgbm_results

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

,Country,Best n_estimators,Best max_depth,Best learning_rate,Best num_leaves,MSE,R2,MAE,RMSE
0,0,50,3,0.01,31,0.004310,0.232706,0.046916,0.065653
1,1,50,3,0.01,31,0.000712,-0.015738,0.018160,0.026680
2,2,50,3,0.01,31,0.000674,-0.046837,0.018933,0.025968
3,3,50,3,0.01,31,0.000121,-0.042519,0.008464,0.010985
4,4,50,3,0.01,31,0.000966,-0.091069,0.020878,0.031088
5,5,50,3,0.01,31,0.001277,0.002839,0.024934,0.035737
6,6,50,3,0.01,31,0.000265,-0.022398,0.011077,0.016268
7,7,50,3,0.01,31,0.001183,-0.039125,0.021061,0.034395
8,8,50,3,0.01,31,0.000418,-0.058700,0.015241,0.020437
9,9,50,3,0.01,31,0.000725,0.003801,0.018311,0.026928


In [ ]:


# Config Time
train_start = datetime.datetime(2000,1,1)
backtest_start = datetime.datetime(2015,1,1)
train_end = backtest_start - pd.DateOffset(days=1)
test_start = backtest_start
test_end = X.index[-1]
train_moving_window_size = pd.DateOffset(months=12)

X_train = X.loc[train_start:train_end]
Y_train = Y.loc[train_start:train_end]
X_test = X.loc[test_start:test_end]
Y_test = Y.loc[test_start:test_end]

# Model
model_choisi = "XGboost"

NameError: name 'Y' is not defined

# 2. Modeling 2

In [ ]:
model_choisi = "linear regression"  # "linear regression", "random forest", "XGBoost" or "lightGBM"

In [ ]:
def run_selected_model(model_choisi, country_data):
    results = []

    for country, df in country_data.items():
        # Déterminer le nom correct de la colonne 'Perf'
        if country == '0':
            perf_col = 'Perf'
        else:
            perf_col = f'Perf.{country}'

        if perf_col in df.columns:
            X = df.drop(perf_col, axis=1)
            y = df[perf_col]

            if X.empty or X.shape[1] == 0:
                print(f"Le pays {country} n'a pas suffisamment de features pour la régression.")
                continue

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

            if model_choisi.lower() == "linear regression":
                model = LinearRegression()
                model.fit(X_train, y_train)
                best_params = {'Model': 'Linear Regression'}

            elif model_choisi.lower() == "random forest":
                param_grid = {
                    'n_estimators': [100, 300, 500],
                    'max_depth': [None, 10, 20],
                    'min_samples_split': [2, 5, 10],
                    'min_samples_leaf': [1, 2, 4]
                }
                model = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
                model.fit(X_train, y_train)
                best_params = model.best_params_
                best_params['Model'] = 'Random Forest'

            elif model_choisi.lower() == "xgboost":
                param_grid = {
                    'n_estimators': [100, 200, 300],
                    'max_depth': [3, 5, 7],
                    'learning_rate': [0.01, 0.1, 0.2],
                    'subsample': [0.8, 0.9, 1.0]
                }
                model = GridSearchCV(XGBRegressor(random_state=42), param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
                model.fit(X_train, y_train)
                best_params = model.best_params_
                best_params['Model'] = 'XGBoost'

            elif model_choisi.lower() == "lightgbm":
                param_grid = {
                    'n_estimators': [50, 100, 200],
                    'max_depth': [3, 5, 10],
                    'learning_rate': [0.01, 0.1, 0.2],
                    'num_leaves': [31, 63, 127],
                    'subsample': [0.8, 0.9, 1.0]
                }
                model = GridSearchCV(LGBMRegressor(random_state=42), param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
                model.fit(X_train, y_train)
                best_params = model.best_params_
                best_params['Model'] = 'LightGBM'

            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mse)

            result = {
                'Country': country,
                'MSE': mse,
                'R2': r2,
                'MAE': mae,
                'RMSE': rmse,
                **best_params
            }
            results.append(result)
        else:
            print(f"Le pays {country} n'a pas de colonne '{perf_col}' pour la prédiction.")

    results_df = pd.DataFrame(results)
    return results_df

results_df = run_selected_model(model_choisi, country_data)
results_df

,Country,MSE,R2,MAE,RMSE,Model
0,0,0.005858,-0.042871,0.054973,0.076541,Linear Regression
1,1,0.000816,-0.163837,0.021209,0.028559,Linear Regression
2,2,0.000770,-0.194844,0.019713,0.027743,Linear Regression
3,3,0.000220,-0.903777,0.011559,0.014844,Linear Regression
4,4,0.000957,-0.080910,0.022855,0.030943,Linear Regression
5,5,0.001200,0.063381,0.028199,0.034635,Linear Regression
6,6,0.000461,-0.782755,0.015491,0.021482,Linear Regression
7,7,0.001647,-0.446421,0.026546,0.040580,Linear Regression
8,8,0.000493,-0.250164,0.015890,0.022208,Linear Regression
9,9,0.000993,-0.364476,0.023150,0.031515,Linear Regression


<div style="background-color: #013220; color: white; padding: 10px;">
    <h1>3. Evaluation</h1>
</div>

In [ ]:
label_pred = model.predict(X_test, Y_test)
f1 = f1_score(y_true=y_test, y_pred=label_pred)



